<a href="https://colab.research.google.com/github/deek2689/CERC_AI/blob/main/Gemini_data_formatting_CERC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [21]:
import pandas as pd
import numpy as np
import json

train_path = 'https://raw.githubusercontent.com/deek2689/CERC_AI/refs/heads/main/train.csv'
test_path = 'https://raw.githubusercontent.com/deek2689/CERC_AI/refs/heads/main/test.csv'
val_path = 'https://raw.githubusercontent.com/deek2689/CERC_AI/refs/heads/main/dev.csv'

train_data = pd.read_csv(train_path)
test_data = pd.read_csv(test_path)
val_data = pd.read_csv(val_path)


In [22]:
train_data.head(10)

,Description,Category
0,"Was walking along crowded street, holding mums...",0
1,This incident took place in the evening.I was ...,0
2,I WAS WAITING FOR THE BUS. A MAN CAME ON A BIK...,1
3,Incident happened inside the train,0
4,I witnessed an incident when a chain was bruta...,0
5,ONE DAY I WAS WALKING AT JORPATI I SAW A BOY T...,0
6,i have heard about indecent people making inde...,0
7,"When I entered the subway it was dark, so I de...",0
8,Sexual Invites,0
9,poor street lighting,0


In [33]:
train_data = train_data.replace({0: False, 1: True})
test_data = test_data.replace({0: False, 1: True})
val_data = test_data = test_data.replace({0: False, 1: True})

<ipython-input-33-76034091e0c3>:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test_data = test_data.replace({0: False, 1: True})


In [34]:
train_data.columns



Index(['Description', 'Category'], dtype='object')

In [35]:
def format_for_gemini(df):
    formatted_data = []

    for _, row in df.iterrows():
        messages = []

        statement = row['Description']
        label = row['Category']

        # System message (Instruction)
        system_message = {
            "role": "system",
            "content": "Classify if the following statement falls under commenting related to sexual harassment. The output must be a single label: 'True' or 'False'."
        }
        messages.append(system_message)

        # User message content containing the statement
        user_message_content = f"STATEMENT: {statement}"
        user_message = {
            "role": "user",
            "content": user_message_content
        }
        messages.append(user_message)

        # Target Label
        model_message = {
            "role": "model",
            "content": label
        }
        messages.append(model_message)

        # Formatting as JSON-compatible dictionary
        formatted_item = {"messages": messages}
        formatted_data.append(formatted_item)

    return formatted_data

Following is a conversation

In [36]:
train_formatted_gemini = format_for_gemini(train_data)
test_formatted_gemini = format_for_gemini(test_data)
val_formatted_gemini = format_for_gemini(val_data)

In [37]:
train_formatted_gemini[0:10]

[{'messages': [{'role': 'system',
    'content': "Classify if the following statement falls under commenting related to sexual harassment. The output must be a single label: 'True' or 'False'."},
   {'role': 'user',
    'content': 'STATEMENT: Was walking along crowded street, holding mums hand, when an elderly man groped butt, I turned to look at h7m and he looked away, and did it again after a while.I was 12 yrs old then.'},
   {'role': 'model', 'content': False}]},
 {'messages': [{'role': 'system',
    'content': "Classify if the following statement falls under commenting related to sexual harassment. The output must be a single label: 'True' or 'False'."},
   {'role': 'user',
    'content': 'STATEMENT: This incident took place in the evening.I was in the metro when two guys started staring.'},
   {'role': 'model', 'content': False}]},
 {'messages': [{'role': 'system',
    'content': "Classify if the following statement falls under commenting related to sexual harassment. The output 

In [45]:
test_formatted_gemini[0:5]

[{'messages': [{'role': 'system',
    'content': "Classify if the following statement falls under commenting related to sexual harassment. The output must be a single label: 'True' or 'False'."},
   {'role': 'user',
    'content': 'STATEMENT: During morning, a woman was walking by and thin guy came around and called her names , and commented on her figure . The woman handled him and abused .'},
   {'role': 'model', 'content': True}]},
 {'messages': [{'role': 'system',
    'content': "Classify if the following statement falls under commenting related to sexual harassment. The output must be a single label: 'True' or 'False'."},
   {'role': 'user',
    'content': "STATEMENT: A man tried to brush his penis off of a woman's shoulder in the bus."},
   {'role': 'model', 'content': False}]},
 {'messages': [{'role': 'system',
    'content': "Classify if the following statement falls under commenting related to sexual harassment. The output must be a single label: 'True' or 'False'."},
   {'rol

In [38]:
print(test_formatted_gemini[2])


{'messages': [{'role': 'system', 'content': "Classify if the following statement falls under commenting related to sexual harassment. The output must be a single label: 'True' or 'False'."}, {'role': 'user', 'content': 'STATEMENT: This happened to a fellow passenger of mine travelling in the metro. He was kicked out of the metro station because he was quotdifferentlyquot dressed.'}, {'role': 'model', 'content': False}]}


In [40]:
len(train_data)

7201

In [41]:
len(val_data)

1701

In [42]:
import json

def save_as_jsonl(data, filename):
    """ Save the dataset in JSON Lines format. """
    with open(filename, 'w', encoding='utf-8') as f:
        for item in data:
            f.write(json.dumps(item) + '\n')

In [43]:
# Saving the datasets
save_as_jsonl(train_formatted_gemini, 'CERC_SafeCity_Commenting_train.jsonl')
save_as_jsonl(test_formatted_gemini, 'CERC_SafeCity_Commenting_test.jsonl')
save_as_jsonl(val_formatted_gemini, 'CERC_SafeCity_Commenting_val.jsonl')

In [ ]:
#save_as_jsonl(validation_data, 'CMV_Dynamic_Gemini_validation.jsonl')